In [ ]:
!pip install wandb

In [ ]:
!pip install torch torchvision torchaudio numpy

## Alternatively, if you intend to run this under ROCm (Only for AMD GPUs)
#!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/rocm6.2.4

In [ ]:
#!wget https://repo.radeon.com/rocm/manylinux/rocm-rel-6.2.3/torch-2.3.0%2Brocm6.2.3-cp310-cp310-linux_x86_64.whl
#!wget https://repo.radeon.com/rocm/manylinux/rocm-rel-6.2.3/torchvision-0.18.0%2Brocm6.2.3-cp310-cp310-linux_x86_64.whl
#!wget https://repo.radeon.com/rocm/manylinux/rocm-rel-6.2.3/pytorch_triton_rocm-2.3.0%2Brocm6.2.3.5a02332983-cp310-cp310-linux_x86_64.whl
#!pip3 install torch-2.3.0+rocm6.2.3-cp310-cp310-linux_x86_64.whl torchvision-0.18.0+rocm6.2.3-cp310-cp310-linux_x86_64.whl pytorch_triton_rocm-2.3.0+rocm6.2.3.5a02332983-cp310-cp310-linux_x86_64.whl

In [ ]:
!wget https://repo.radeon.com/rocm/manylinux/rocm-rel-6.3.1/torch-2.3.0%2Brocm6.3.1-cp312-cp312-linux_x86_64.whl
!wget https://repo.radeon.com/rocm/manylinux/rocm-rel-6.3.1/torchvision-0.18.0%2Brocm6.3.1-cp312-cp312-linux_x86_64.whl
!wget https://repo.radeon.com/rocm/manylinux/rocm-rel-6.3.1/pytorch_triton_rocm-2.3.0%2Brocm6.3.1.5a02332983-cp312-cp312-linux_x86_64.whl


In [ ]:
!pip3 install torch-2.3.0+rocm6.3.1-cp312-cp312-linux_x86_64.whl torchvision-0.18.0+rocm6.3.1-cp312-cp312-linux_x86_64.whl pytorch_triton_rocm-2.3.0+rocm6.3.1.5a02332983-cp312-cp312-linux_x86_64.whl

In [ ]:
import wandb

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="convomotional",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.0001,
    "architecture": "CNN",
    "dataset": "yousefmohamed20/sentiment-images-classifier",
    "epochs": 30,
    }
)

In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import random_split, DataLoader

# Update the transform for both training and validation
transform = transforms.Compose([
    transforms.Resize(299),  # Resize to Inception V3's required size
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225])
])

batch_size=32
# Update your dataset and dataloader initialization with the new transform
train_dataset = datasets.ImageFolder(root='emotions_dataset/emotions_dataset_cropped_faces', transform=transform)
val_dataset = datasets.ImageFolder(root='emotions_dataset/emotions_dataset_cropped_faces', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# Optional: Print dataset sizes for confirmation
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

# Common dependencies

In [20]:
import torch.nn as nn
import torchvision.models as models

In [21]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        val_accuracy = 100 * correct / total

        wandb.log({"train_loss": train_loss, "val_loss": val_loss, "val_accuracy": val_accuracy})
        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, "
              f"Val Accuracy: {val_accuracy:.2f}%")

    return model


In [22]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")


In [23]:
# Import Inception V3
from torchvision.models import inception_v3, Inception_V3_Weights

# Load and modify the model
def get_inception_model(num_classes):
    model = inception_v3(weights=Inception_V3_Weights.DEFAULT)
    model.aux_logits = False  # Disable auxiliary output
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

In [ ]:
import os
# Parameters
os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.3"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(device)
num_classes = len(train_dataset.classes)  # Automatically detect number of classes
num_epochs = 10
learning_rate = 0.0001

# Model, Loss, Optimizer
model = get_inception_model(num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Train the model
os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.3"
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

In [ ]:
# Test the model
evaluate_model(trained_model, test_loader, device)
wandb.finish()